In [1]:
import sys
import torch
import json
from pathlib import Path
sys.path.append('..')

from cgol.generator.uniform_density_generator import UniformDensityGenerator
from cgol.simulator.minimal_architecture_simulator import MinimalArchitectureSimulator
from cgol.dataloader.dataloader_2 import Dataloader2

from solutions.nca.nca_feedback_model import NCAFeedbackModel

In [2]:
model_device = torch.device('cuda')
dtype = torch.float

checkpoints_base_path = Path('../solutions/nca')

models = [
    {
        'name': 'nca_feedback_train',
        'run_id': '3oxz77pn',
        'trainer': 'nca_feedback_train.ipynb',
        'checkpoints_folder': checkpoints_base_path/'nca_feedback_train',
        'create_model': lambda: NCAFeedbackModel(device=model_device, dtype=dtype),
        'init_model': lambda model, checkpoint: model.load_state_dict(checkpoint['model_state'])
    }
]

In [3]:
with torch.no_grad():
    preprocess_device = torch.device('cpu')

    generator = UniformDensityGenerator(1234, preprocess_device, dtype)
    preprocess_simulator = MinimalArchitectureSimulator(preprocess_device, dtype)
    eval_simulator = MinimalArchitectureSimulator(model_device, dtype)

    batch_size = 100
    n_samples = 10000
    width = 20
    height = 20
    n_solving_steps = 400

    result_file_path = Path(f'{width}x{height}_uniform_{n_samples}_{batch_size}.json')

    result = {}
    if result_file_path.exists():
        with result_file_path.open() as result_file:
            result = json.load(result_file)
    for model_dict in models:
        print(model_dict['name'])
        for checkpoint_path in model_dict['checkpoints_folder'].iterdir():
            checkpoint = torch.load(checkpoint_path)
            result_name = f"{model_dict['name']}-{checkpoint['run_state']['step']}"
            print(result_name)
            print(checkpoint['run_state']['accuracy_micro'])
            print(checkpoint['run_state']['accuracy_macro'])
            model_result = {}
            if result_name in result.keys():
                model_result = result[result_name]
            else:
                model: NCAFeedbackModel = model_dict['create_model']()
                model.eval()
                model_dict['init_model'](model, checkpoint)
                model.eval()

                #batch_size = model_dict['batch_size']
                dataloader = Dataloader2(generator, preprocess_simulator,
                                        batch_size, width, height, 
                                        preprocess_device, model_device, dtype,
                                        0.1, 150)

                n_samples_left = n_samples
                best_solving_steps_sum = 0
                initial_state_n_cells_correct = 0
                initial_state_n_samples_correct = 0
                end_state_n_cells_correct = 0
                end_state_n_samples_correct = 0

                state_solving_steps = 0

                while n_samples_left > 0:
                    batch = next(dataloader)
                    batch = batch[:min(batch_size, n_samples_left)]
                    n_samples_left -= batch.shape[1]
                    print(n_samples_left)

                    end_state_target = batch[0]
                    initial_state_target = batch[1]
                    model_input = model.init_model_input(end_state_target)

                    best_initial_state_cells_correct = torch.zeros(batch[0].shape, device=model_device, dtype=torch.bool)
                    best_end_state_cells_correct = torch.zeros(batch[0].shape, device=model_device, dtype=torch.bool)
                    best_solving_step = torch.zeros((batch[0].shape[0], ), device=model_device, dtype=dtype)
                    best_end_state_n_cells_in_sample_correct = torch.zeros((batch[0].shape[0], ), device=model_device, dtype=torch.long)

                    for i_solving_step in range(1, n_solving_steps+1):
                        model_output = model(model_input)
                        output = model.output_batch_from_model_output(model_output)

                        initial_state_predicted = (output >= 0.5)*1.
                        end_state_predicted = eval_simulator.step_batch_tensor(initial_state_predicted)
                        model_input = model.model_input_from_model_output(model_output, end_state_target, (end_state_predicted!=end_state_target)*1.)

                        solving_step_cells_correct = end_state_target == end_state_predicted
                        solving_step_n_cells_in_sample_correct = solving_step_cells_correct.sum((-1, -2))
                        new_best = best_end_state_n_cells_in_sample_correct < solving_step_n_cells_in_sample_correct
                        best_solving_step[new_best] = i_solving_step
                        best_initial_state_cells_correct[new_best] = (initial_state_target == initial_state_predicted)[new_best]
                        best_end_state_cells_correct[new_best] = solving_step_cells_correct[new_best]
                        best_end_state_n_cells_in_sample_correct[new_best] = solving_step_n_cells_in_sample_correct[new_best]

                    best_solving_steps_sum += best_solving_step.sum().item()
                    initial_state_n_cells_correct += best_initial_state_cells_correct.sum().item()
                    initial_state_n_samples_correct += best_initial_state_cells_correct.all((-1,-2)).sum().item()
                    end_state_n_cells_correct += best_end_state_cells_correct.sum().item()
                    end_state_n_samples_correct += best_end_state_cells_correct.all((-1, -2)).sum().item()

                n_cells = n_samples * width * height
                model_result['mean_solving_steps'] = best_solving_steps_sum / n_samples
                model_result['initial_state_accuracy_micro'] = initial_state_n_cells_correct / n_cells
                model_result['initial_state_accuracy_macro'] = initial_state_n_samples_correct / n_samples
                model_result['end_state_accuracy_micro'] = end_state_n_cells_correct / n_cells
                model_result['end_state_accuracy_macro'] = end_state_n_samples_correct / n_samples
            result[result_name] = model_result

    result_json =json.dumps(result, indent=2)
    with open(result_file_path, 'w') as result_file:
        result_file.write(result_json)
    print(result_json)

nca_feedback_train
nca_feedback_train-4473412
tensor(0.9159, device='cuda:0')
tensor(0.0020, device='cuda:0')
9900
9800
9700
9600
9500
9400
9300
9200
9100
9000
8900
8800
8700
8600
8500
8400
8300
8200
8100
8000
7900
7800
7700
7600
7500
7400
7300
7200
7100
7000
6900
6800
6700
6600
6500
6400
6300
6200
6100
6000
5900
5800
5700
5600
5500
5400
5300
5200
5100
5000
4900
4800
4700
4600
4500
4400
4300
4200
4100
4000
3900
3800
3700
3600
3500
3400
3300
3200
3100
3000
2900
2800
2700
2600
2500
2400
2300
2200
2100
2000
1900
1800
1700
1600
1500
1400
1300
1200
1100
1000
900
800
700
600
500
400
300
200
100
0
nca_feedback_train-4173176
tensor(0.9206, device='cuda:0')
tensor(0.0020, device='cuda:0')
9900
9800
9700
9600
9500
9400
9300
9200
9100
9000
8900
8800
8700
8600
8500
8400
8300
8200
8100
8000
7900
7800
7700
7600
7500
7400
7300
7200
7100
7000
6900
6800
6700
6600
6500
6400
6300
6200
6100
6000
5900
5800
5700
5600
5500
5400
5300
5200
5100
5000
4900
4800
4700
4600
4500
4400
4300
4200
4100
4000
3900
3800
3